In [4]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from feedback_grape.fgrape import optimize_pulse, evaluate_on_longer_time
from helpers import (
    init_grape_protocol,
    init_fgrape_protocol,
    test_implementations,
    generate_random_discrete_state,
    generate_random_bloch_state
)
from tqdm import tqdm
import jax
import jax.numpy as jnp
import numpy as np
from library.utils.FgResult_to_dict import FgResult_to_dict
import json

test_implementations()

jax.config.update("jax_enable_x64", True)

In [5]:
# Physical parameters
# (attention! #elements in density matrix grow as 4^N_chains)
N_chains = 3 # Number of parallel chains to simulate
gamma = 0.05 # Decay constant

# Training and evaluation parameters
training_params = {
    "N_samples": 10, # Number of random initial states to sample
    "N_training_iterations": 10000, # Number of training iterations
    "learning_rate": 0.01, # Learning rate
    "convergence_threshold": 1e-6,
    "batch_size": 8,
    "eval_batch_size": 16,
    "evaluation_time_steps": 100,
}
N_parallel_threads = 4 # Number of parallel threads for training
generate_state = generate_random_bloch_state

# Parameters to test

#num_time_steps : Number of time steps in the control pulse
#lut_depth : Depth of the lookup table for feedback
#reward_weights: Weights for the reward at each time step. Default only weights last timestep [0, 0, ... 0, 1]

experiments = [ # (timesteps, lut_depth, reward_weights, noise_level)
    (t, 2, [0]*(t-1)+[1], noise)  for t in [9,10,11,12,13,14,15] for noise in [0.0, 0.5, 1.0, float('inf')]
]

experiments_format = [ # Format of variables as tuples (name, type, required by grape, required by lut, required by rnn)
    ("t", int, True, True, True), # number timesteps (or segments)
    ("l", int, False, True, False), # LUT depth
    ("w", list, True, True, True), # reward weights
    ("noise", float, True, True, True), # noise level
]

# Check if experiment parameters are in specified format
for params in experiments:
    assert len(params) == len(experiments_format), "Experiment parameters length mismatch."
    for i, (param, (name, expected_type, req_grape, req_lut, req_rnn)) in enumerate(zip(params, experiments_format)):
        assert type(name) == str, "Parameter name must be a string."
        assert type(req_grape) == bool, "Requirement flags must be boolean."
        assert type(req_lut) == bool, "Requirement flags must be boolean."
        assert type(req_rnn) == bool, "Requirement flags must be boolean."
        assert type(param) == expected_type, f"Parameter '{name}' at index {i} must be of type {expected_type.__name__}."
        if type(param) == list:
            for j, item in enumerate(param):
                assert type(item) == int and 0 <= item <= 9, f"Item at index {j} in parameter '{name}' must be of type int and between 0 and 9."

def generate_param_paths(params, model_type):
    assert model_type in ["grape", "lut", "rnn"], "Invalid model type."

    parts = []
    for (name, expected_type, req_grape, req_lut, req_rnn), param in zip(experiments_format, params):
        required = (model_type == "grape" and req_grape) or (model_type == "lut" and req_lut) or (model_type == "rnn" and req_rnn)
        if required:
            if expected_type == list:
                param_str = "".join(str(x) for x in param)
            else:
                param_str = str(param)
            parts.append(f"{name}={param_str}")

    pstr = f"{model_type}_" + "_".join(parts)
    return (
        "./optimized_architectures/" + pstr + ".json",
        "./optimized_architectures/" + pstr + "_training_data.json",
        "./evaluation_results/" + pstr + ".npz"
    )

In [6]:
from concurrent.futures import ThreadPoolExecutor

def run_experiment(params):
    num_time_steps, lut_depth, reward_weights, noise_level = params
    print(f"Evaluating {params}")
    noisy_state = lambda key: generate_state(key, N_chains=N_chains, noise_level=noise_level)
    pure_state = lambda key: generate_state(key, N_chains=N_chains, noise_level=0.0)

    # Only test LUTs in this example
    path1, path2, path3 = generate_param_paths(params, "lut")
    if os.path.exists(path1):
        print(f"LUT already trained, skipping training.")
    else:
        print("Training LUT")

        best_result = None
        best_result_fidelities = None
        fidelities_each = []
        for s in tqdm(range(training_params["N_samples"])):
            system_params = init_fgrape_protocol(jax.random.PRNGKey(s), N_chains, gamma)

            # Train LUT
            result = optimize_pulse(
                U_0=noisy_state,
                C_target=pure_state,
                system_params=system_params,
                num_time_steps=num_time_steps,
                lut_depth=lut_depth,
                reward_weights=reward_weights,
                mode="lookup",
                goal="fidelity",
                max_iter=training_params["N_training_iterations"],
                convergence_threshold=training_params["convergence_threshold"],
                learning_rate=training_params["learning_rate"],
                evo_type="density",
                batch_size=training_params["batch_size"],
                eval_batch_size=1, # This evaluation is discarded
            )

            eval_result = evaluate_on_longer_time( # Evaluate on longer time and choose best LUT accordingly
                U_0 = pure_state,
                C_target = pure_state,
                system_params = system_params,
                optimized_trainable_parameters = result.optimized_trainable_parameters,
                num_time_steps = training_params["evaluation_time_steps"],
                evo_type = "density",
                goal = "fidelity",
                eval_batch_size = training_params["eval_batch_size"],
                mode = "lookup",
            )

            fidelities = eval_result.fidelity_each_timestep

            if best_result is None or np.mean(fidelities) > np.mean(best_result_fidelities):
                best_result = result
                best_result_fidelities = fidelities

            fidelities_each.append(float(np.mean(fidelities)))

        np.savez(path3, fidelities_lut=np.array(best_result_fidelities))

        with open(path1, "w") as f:
            json.dump(FgResult_to_dict(best_result), f)

        with open(path2, "w") as f:
            training_data = {
                "fidelities_each_sample": fidelities_each,
                "average_fidelity": float(np.mean(fidelities_each)),
                "training_params": training_params,
            }
            json.dump(training_data, f)

# Run experiments in parallel
if N_parallel_threads > 1:
    with ThreadPoolExecutor(max_workers=N_parallel_threads) as executor:
        executor.map(run_experiment, experiments)
else:
    for params in experiments:
        run_experiment(params)

# Play a sound when done
os.system('say "done."')

Evaluating (9, 2, [0, 0, 0, 0, 0, 0, 0, 0, 1], 0.0)Evaluating (9, 2, [0, 0, 0, 0, 0, 0, 0, 0, 1], 0.5)

Training LUT
Training LUT
Evaluating (9, 2, [0, 0, 0, 0, 0, 0, 0, 0, 1], 1.0)
Training LUT
Evaluating (9, 2, [0, 0, 0, 0, 0, 0, 0, 0, 1], inf)
Training LUT


  0%|          | 0/10 [00:00<?, ?it/s]




 10%|█         | 1/10 [05:08<46:17, 308.63s/it]


 20%|██        | 2/10 [10:27<41:58, 314.83s/it]


 30%|███       | 3/10 [15:50<37:09, 318.46s/it]


 40%|████      | 4/10 [21:14<32:04, 320.74s/it]


 50%|█████     | 5/10 [26:33<26:39, 319.93s/it]


 60%|██████    | 6/10 [31:52<21:19, 319.83s/it]


 70%|███████   | 7/10 [37:12<15:58, 319.65s/it]


 80%|████████  | 8/10 [42:31<10:38, 319.37s/it]


 90%|█████████ | 9/10 [47:53<05:20, 320.29s/it]


100%|██████████| 10/10 [53:17<00:00, 319.71s/it]


Evaluating (10, 2, [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 0.0)
Training LUT


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [53:45<00:00, 322.56s/it]


...

Evaluating (15, 2, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 1.0)
Training LUT





100%|██████████| 10/10 [1:27:57<00:00, 527.75s/it]


Evaluating (15, 2, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], inf)
Training LUT



 10%|█         | 1/10 [09:25<1:24:48, 565.44s/it]



 20%|██        | 2/10 [18:54<1:15:40, 567.57s/it]

 30%|███       | 3/10 [24:21<53:23, 457.61s/it]  


 40%|████      | 4/10 [33:51<50:13, 502.18s/it]


 50%|█████     | 5/10 [43:27<44:02, 528.53s/it]


 70%|███████   | 7/10 [55:24<20:40, 413.45s/it]


 80%|████████  | 8/10 [1:04:53<15:25, 462.77s/it]


 90%|█████████ | 9/10 [1:14:23<08:16, 496.47s/it]


100%|██████████| 10/10 [1:23:53<00:00, 503.32s/it]




100%|██████████| 10/10 [1:33:28<00:00, 560.86s/it]


100%|██████████| 10/10 [1:32:55<00:00, 557.54s/it]


0